The city of New York does restaurant inspections and assigns a grade.
Inspections data the last 4 years are available
[here](https://s3.amazonaws.com/thedataincubator/coursedata/nyc_inspection_data.zip).

The file `RI_Webextract_BigApps_Latest.xls` contains a description of each of
the datafiles.  Take a look and then load the csv formatted `*.txt` files into
Postgresql into five tables:
1. `actions`
2. `cuisines`
3. `violations`
4. `grades` (from `WebExtract.txt`)
5. `boroughs` (from `RI_Webextract_BigApps_Latest.xls`)

# Hints
1. It is recommended to use sqlite3 for this project. Postgresql can work but
   will be more difficult to set up properly on Digital Ocean. If you do use
   sqlite, in order to do mathematical calculations like square root, you will
   need to compile and install the extension described on the
   [wiki](https://sites.google.com/a/thedataincubator.com/the-data-incubator-wiki/course-information-and-logistics/getting-started/setup).
2. Sqlite3 has a convenient .import function which can create tables from .csv .
   You may want to set up your databases using the sqlite shell, then use Python
   just to run the SELECT queries.

   Postgresql has an equivalent [`\copy`
   command](http://www.postgresql.org/docs/9.2/static/app-psql.html#APP-PSQL-META-COMMANDS-COPY)
   that can both save and load files in various formats.  It is a convenience
   wrapper for the [`copy`
   command](http://www.postgresql.org/docs/9.2/static/sql-copy.html) but
   behaves better (e.g. relative paths).
3. The files may contain malformatted text.  Unfortunately, this is all too
   common.  As a stop gap, remember that `iconv` is a unix utility that can
   convert files between different text encodings.
4. For more sophisticated needs, a good strategy is to write simple python
   scripts that will reparse files.  For example, commas (',') within a single
   field will trick many csv parsers into breaking up the field.  Write a
   python script that converts these 'inadvertent' delimiters into semicolons
   (';').

# Questions

## score_by_zipcode
Return a list of tuples of the form:
```
(zipcode, mean score, standard error, number of violations)
```
for each of the 184 zipcodes in the city with over 100 violations. Sort the
list in ascending order by mean score. You can read more about standard error
on [wikipedia](http://en.wikipedia.org/wiki/Standard_error).

**Note:** There is an interesting discussion here about what the mean score
*means* in this dataset. Think about what we're actually calculating - 
does it represent what we're trying to understand about these zipcodes?

In other words, what is the difference between the question we're trying
to answer with this analysis intuitively and the question we're actually
answering?

If we were to do a different calculation, what biases would we run into?
How should we deal with this kind of situation as data scientists? 

**Checkpoint**
Total entries in valid zipcodes: 531,126

## score_by_map
The above are not terribly enlightening.  Use [CartoDB](http://cartodb.com/)
to produce a map of average scores by zip code.  You can sign up for a free
trial.

You will have to use their wizard to plot the data by
[zipcode](http://docs.cartodb.com/cartodb-editor.html#geocoding-data).  Then
use the "share" button to return a link to a short URL beginning with
"http://cdb.io/".

**For fun:** How do JFK, Brighton Beach, Liberty Island (home of the Statue of
Liberty), Financial District, Chinatown, and Coney Island fare?

## score_by_borough
Return a list of tuples of the form:
    ```
    (borough, mean score, stderr, number of violations)
    ```
for each of the city's five boroughs. Sort the list in ascending order by grade.

**Hint**: you will have to perform a join with the `boroughs` table.

**Checkpoint**
Total entries in valid boroughs: 531,832

## score_by_cuisine
Return a list of the 75 tuples of the form
    ```
    (cuisine, mean score, stderr, number of violations)
    ```
for each of the 75 cuisine types with at least 100 inspections. Sort the list 
in ascending order by score. Are the least sanitary and most sanitary cuisine
types surprising?

**Note:** It's interesting to think again about what this analysis is trying
to say and how it differs from the analysis by zipcode. How should this
affect the calculation in your opinion?

**Checkpoint**
Total entries from valid cuisines: 531,529

## violation_by_cuisine
Which cuisines tend to have a disproportionate number of what which violations?
Answering this question isn't easy becuase you have to think carefully about
normalizations.

1. More popular cuisine categories will tend to have more violations just
   becuase they represent more restaurants.
2. Similarly, some violations are more common.  For example, knowing that
   "Equipment not easily movable or sealed to floor" is a common violation for
   Chinese restuarants is not particularly helpful when it is a common
   violation for all restaurants.

The right quantity is to look at is the conditional probability of a specific
type of violation given a specific cuisine type and divide it by the
unconditional probability of the violation for the entire population. Taking
this ratio gives the right answer.  Return the 20 highest ratios of the form:
    ```
    ((cuisine, violation), ratio, count)
    ```

**Hint**:
1. You might want to check out this [Stackoverflow
   post](http://stackoverflow.com/questions/972877/calculate-frequency-using-sql).
2. The definition of a violation changes with time.  For example, 10A can mean
   two different things "Toilet facility not maintained ..." or "Vermin or
   other live animal present ..." when things were prior to 2003. To deal with
   this, you should limit your analysis to violation codes with end date after
   Jan 1, 2014. (This end date refers to the validity time ranges in
   Violation.txt).
3. The ratios don't mean much when the number of violations of a given type and
   for a specific category are not large (why not?).  Be sure to filter these
   out.  We chose 100 as our cutoff.

**Checkpoint**:
Top 20 ratios mean: 1.95724597861122


# Cuisine violation

In [329]:
vi = pd.read_csv('Violation_corrected.txt', skiprows=1)

In [331]:
vi

STARTDATE    ENDDATE CRITICALFLAG VIOLATIONCODE  \
0    1901-01-01 00:00:00 2003-03-23            Y           01A   
1    2003-03-24 00:00:00 2005-02-17            Y           01A   
2    2005-02-18 00:00:00 2007-06-30            Y           01A   
3    2007-07-01 00:00:00 2008-06-30            Y           01A   
4    2008-07-01 00:00:00 2009-08-01            Y           01A   
5    1901-01-01 00:00:00 2003-03-23            Y           01B   
6    2003-03-24 00:00:00 2005-02-17            Y           01B   
7    2005-02-18 00:00:00 2007-06-30            Y           01B   
8    2007-07-01 00:00:00 2008-06-30            Y           01B   
9    2008-07-01 00:00:00 2009-08-01            Y           01B   
10   1901-01-01 00:00:00 2003-03-23            Y           01C   
11   2003-03-24 00:00:00 2005-02-17            Y           01C   
12   2005-02-18 00:00:00 2007-06-30            Y           01C   
13   2007-07-01 00:00:00 2008-06-30            Y           01C   
14   2008-07-01 00:00:00 2009-08-01            Y           01C   
15   1901-01-01 00:00:00 2003-03-23            Y           01D   
16   2003-03-24 00:00:00 2005-02-17            Y           01D   
17   2005-02-18 00:00:00 2007-06-30            Y           01D   
18   2007-07-01 00:00:00 2008-06-30            Y           01D   
19   2008-07-01 00:00:00 2009-08-01            Y           01D   
20   1901-01-01 00:00:00 2003-03-23            Y           01E   
21   2003-03-24 00:00:00 2005-02-17            Y           01E   
22   2005-02-18 00:00:00 2007-06-30            Y           01E   
23   2007-07-01 00:00:00 2008-06-30            Y           01E   
24   2008-07-01 00:00:00 2009-08-01            Y           01E   
25   1901-01-01 00:00:00 2003-03-23            Y           01F   
26   2003-03-24 00:00:00 2005-02-17            Y           01F   
27   2005-02-18 00:00:00 2007-06-30            Y           01F   
28   2007-07-01 00:00:00 2008-06-30            Y           01F   
29   2008-07-01 00:00:00 2009-08-01            Y           01F   
..                   ...        ...          ...           ...   
689  2010-07-26 00:00:00 2099-12-31            N           16F   
690  2009-08-02 00:00:00 2010-07-25            N           18A   
691  2010-07-26 00:00:00 2099-12-31            N           18A   
692  2009-08-02 00:00:00 2010-07-25            N           18B   
693  2010-07-26 00:00:00 2099-12-31            N           18B   
694  2009-08-02 00:00:00 2010-07-25            N           18C   
695  2010-07-26 00:00:00 2099-12-31            N           18C   
696  2009-08-02 00:00:00 2010-07-25            N           18D   
697  2010-07-26 00:00:00 2099-12-31            N           18D   
698  2009-08-02 00:00:00 2010-07-25            N           18E   
699  2010-07-26 00:00:00 2099-12-31            N           18E   
700  2009-08-02 00:00:00 2010-07-25            N           18F   
701  2010-07-26 00:00:00 2099-12-31            N           18F   
702  2009-08-02 00:00:00 2010-07-25            N           18G   
703  2010-07-26 00:00:00 2099-12-31            N           18G   
704  2009-08-02 00:00:00 2010-07-25            N           18H   
705  2010-07-26 00:00:00 2099-12-31            N           18H   
706  2009-08-02 00:00:00 2010-07-25            N           18I   
707  2010-07-26 00:00:00 2099-12-31            N           20A   
708  2010-07-26 00:00:00 2099-12-31            N           20B   
709  2010-07-26 00:00:00 2099-12-31            N           20C   
710  2010-07-26 00:00:00 2099-12-31            N           20D   
711  2010-07-26 00:00:00 2099-12-31            N           20E   
712  2010-07-26 00:00:00 2099-12-31            N           20F   
713  2010-07-26 00:00:00 2099-12-31            N           22A   
714  2010-07-26 00:00:00 2099-12-31            N           22B   
715  2010-07-26 00:00:00 2099-12-31            N           22C   
716  2010-07-26 00:00:00 2099-12-31            N           22E   
717  2009-08-02 00:00:00 2010-07-25            N       

In [323]:
vi['ENDDATE']

0                  ENDDATE
1      2003-03-23 00:00:00
2      2005-02-17 00:00:00
3      2007-06-30 00:00:00
4      2008-06-30 00:00:00
5      2009-08-01 00:00:00
6      2003-03-23 00:00:00
7      2005-02-17 00:00:00
8      2007-06-30 00:00:00
9      2008-06-30 00:00:00
10     2009-08-01 00:00:00
11     2003-03-23 00:00:00
12     2005-02-17 00:00:00
13     2007-06-30 00:00:00
14     2008-06-30 00:00:00
15     2009-08-01 00:00:00
16     2003-03-23 00:00:00
17     2005-02-17 00:00:00
18     2007-06-30 00:00:00
19     2008-06-30 00:00:00
20     2009-08-01 00:00:00
21     2003-03-23 00:00:00
22     2005-02-17 00:00:00
23     2007-06-30 00:00:00
24     2008-06-30 00:00:00
25     2009-08-01 00:00:00
26     2003-03-23 00:00:00
27     2005-02-17 00:00:00
28     2007-06-30 00:00:00
29     2008-06-30 00:00:00
              ...         
690    2099-12-31 00:00:00
691    2010-07-25 00:00:00
692    2099-12-31 00:00:00
693    2010-07-25 00:00:00
694    2099-12-31 00:00:00
695    2010-07-25 00:00:00
6

In [330]:
vi['ENDDATE'] = pd.to_datetime(vi['ENDDATE'], format='%Y-%m-%d %H:%M:%S')

In [332]:
from time import strptime
thr_date_str = 'January 1, 2014'
thr_date = strptime(thr_date_str, "%B %d, %Y")
vi_ok_indices = vi['ENDDATE'] > thr_date_str
vi_ok = vi.loc[vi_ok_indices,['VIOLATIONCODE']]

In [333]:
vi_ok['']

VIOLATIONCODE
39            02A
46            02B
53            02C
60            02D
67            02E
74            02F
80            02G
86            02H
92            02I
94            02J
101           03A
108           03B
115           03C
121           03D
127           03E
133           03F
139           03G
147           04A
154           04B
161           04C
168           04D
175           04E
182           04F
189           04G
195           04H
201           04I
207           04J
213           04K
219           04L
225           04M
..            ...
661           15P
662           15Q
663           15R
664           15S
665           15T
669           16A
673           16B
677           16C
681           16D
685           16E
689           16F
691           18A
693           18B
695           18C
697           18D
699           18E
701           18F
703           18G
705           18H
707           20A
708           20B
709           20C
710           20D
711           20E
712           20F
713           22A
714           22B
715           22C
716           22E
718           99B

[112 rows x 1 columns]

In [337]:
vi_ok['VIOLATIONCODE'].values

array(['02A', '02B', '02C', '02D', '02E', '02F', '02G', '02H', '02I',
       '02J', '03A', '03B', '03C', '03D', '03E', '03F', '03G', '04A',
       '04B', '04C', '04D', '04E', '04F', '04G', '04H', '04I', '04J',
       '04K', '04L', '04M', '04N', '04O', '05A', '05B', '05C', '05D',
       '05E', '05F', '05G', '05H', '05I', '06A', '06B', '06C', '06D',
       '06E', '06F', '06G', '06H', '06I', '07A', '08A', '08B', '08C',
       '09A', '09B', '09C', '10A', '10B', '10C', '10D', '10E', '10F',
       '10G', '10H', '10I', '10J', '15A', '15B', '15C', '15D', '15E',
       '15F', '15G', '15H', '15I', '15J', '15K', '15L', '15M', '15N',
       '15O', '15P', '15Q', '15R', '15S', '15T', '16A', '16B', '16C',
       '16D', '16E', '16F', '18A', '18B', '18C', '18D', '18E', '18F',
       '18G', '18H', '20A', '20B', '20C', '20D', '20E', '20F', '22A',
       '22B', '22C', '22E', '99B'], dtype=object)

In [310]:
tt1['RECORDDATE'] = pd.to_datetime(tt1['RECORDDATE'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
P(V|C)/P(V)

In [344]:
tt1_ok_indices = [val in vi_ok['VIOLATIONCODE'].values for val in tt1['VIOLCODE'].values]

array(['10F', '06A', '10B', ..., nan, nan, nan], dtype=object)

In [350]:
len(tt1_ok_indices)

531935

In [351]:
tt1_ok = tt1.loc[tt1_ok_indices]

In [360]:
pc = pd.DataFrame(tt1_ok.groupby('VIOLCODE').size())
pc.columns=['count']
pc_fl = pc[pc['count']>=100].sort('count', ascending = False)
pc_fl['VIOLATION_PROB'] = pc_fl['count'] / sum(pc_fl['count'])


count  VIOLATION_PROB
VIOLCODE                       
10F       66420        0.127804
08A       49564        0.095370
02G       45819        0.088164
04L       38306        0.073708
10B       31331        0.060286
06D       30966        0.059584
06C       25370        0.048816
02B       24523        0.047187
04N       21508        0.041385
04H       15954        0.030698
04M       14568        0.028031
06E       13741        0.026440
06F       10016        0.019273
06A        9381        0.018051
04A        9264        0.017826
10H        8727        0.016792
09C        6806        0.013096
04J        6272        0.012068
08C        6240        0.012007
05D        5960        0.011468
10A        5860        0.011276
16B        5852        0.011260
10D        4979        0.009580
04C        4726        0.009094
22C        4401        0.008468
09B        4364        0.008397
10I        3898        0.007500
20F        3775        0.007264
10E        3536        0.006804
04K        3306        0.006361
...         ...             ...
08B        2400        0.004618
99B        2257        0.004343
20D        1945        0.003743
22A        1746        0.003360
16A         916        0.001763
20A         884        0.001701
05H         844        0.001624
05F         827        0.001591
03A         593        0.001141
02C         550        0.001058
04O         541        0.001041
02A         498        0.000958
04D         496        0.000954
10C         479        0.000922
20E         476        0.000916
03C         474        0.000912
15J         464        0.000893
15I         453        0.000872
22B         396        0.000762
15K         391        0.000752
03B         331        0.000637
04E         324        0.000623
15S         298        0.000573
16C         188        0.000362
07A         155        0.000298
05E         151        0.000291
18D         137        0.000264
10G         115        0.000221
06G         114        0.000219
05A         111        0.000214

[65 rows x 2 columns]

In [400]:
sum(pc_fl['count'])

519702

In [398]:
pc_vl_new = pc_fl.copy()
pc_vl_new['VIOLCODE'] = pc_vl_new.index
pc_vl_new = pd.concat([pc_vl_new] * len(pc_cz_fl))
pc_vl_new.reset_index(drop=True, inplace=True)
pc_vl_new['CUISINECODE'] = cz_reps
pc_vl_new['CUISINE_PROB'] = cz_prob_reps
pc_vl_new

count  VIOLATION_PROB VIOLCODE  CUISINECODE  CUISINE_PROB
0     66420        0.127804      10F            3      0.240925
1     49564        0.095370      08A            3      0.240925
2     45819        0.088164      02G            3      0.240925
3     38306        0.073708      04L            3      0.240925
4     31331        0.060286      10B            3      0.240925
5     30966        0.059584      06D            3      0.240925
6     25370        0.048816      06C            3      0.240925
7     24523        0.047187      02B            3      0.240925
8     21508        0.041385      04N            3      0.240925
9     15954        0.030698      04H            3      0.240925
10    14568        0.028031      04M            3      0.240925
11    13741        0.026440      06E            3      0.240925
12    10016        0.019273      06F            3      0.240925
13     9381        0.018051      06A            3      0.240925
14     9264        0.017826      04A            3      0.240925
15     8727        0.016792      10H            3      0.240925
16     6806        0.013096      09C            3      0.240925
17     6272        0.012068      04J            3      0.240925
18     6240        0.012007      08C            3      0.240925
19     5960        0.011468      05D            3      0.240925
20     5860        0.011276      10A            3      0.240925
21     5852        0.011260      16B            3      0.240925
22     4979        0.009580      10D            3      0.240925
23     4726        0.009094      04C            3      0.240925
24     4401        0.008468      22C            3      0.240925
25     4364        0.008397      09B            3      0.240925
26     3898        0.007500      10I            3      0.240925
27     3775        0.007264      20F            3      0.240925
28     3536        0.006804      10E            3      0.240925
29     3306        0.006361      04K            3      0.240925
...     ...             ...      ...          ...           ...
5430   2400        0.004618      08B           16      0.000004
5431   2257        0.004343      99B           16      0.000004
5432   1945        0.003743      20D           16      0.000004
5433   1746        0.003360      22A           16      0.000004
5434    916        0.001763      16A           16      0.000004
5435    884        0.001701      20A           16      0.000004
5436    844        0.001624      05H           16      0.000004
5437    827        0.001591      05F           16      0.000004
5438    593        0.001141      03A           16      0.000004
5439    550        0.001058      02C           16      0.000004
5440    541        0.001041      04O           16      0.000004
5441    498        0.000958      02A           16      0.000004
5442    496        0.000954      04D           16      0.000004
5443    479        0.000922      10C           16      0.000004
5444    476        0.000916      20E           16      0.000004
5445    474        0.000912      03C           16      0.000004
5446    464        0.000893      15J           16      0.000004
5447    453        0.000872      15I           16      0.000004
5448    396        0.000762      22B           16      0.000004
5449    391        0.000752      15K           16      0.000004
5450    331        0.000637      03B           16      0.000004
5451    324        0.000623      04E           16      0.000004
5452    298        0.000573      15S           16      0.000004
5453    188        0.000362      16C           16      0.000004
5454    155        0.000298      07A           16      0.000004
5455    151        0.000291      05E           16      0.000004
5456    137        0.000264      18D           16      0.000004
5457    115        0.000221      10G           16      0.000004
5458    114        0.000219      06G           16      0.000004
5459    111        0.000214      05A           16      0.000004

[5460 rows x 5 columns]

In [399]:
tt1_ok

CAMIS                                   DBA  BORO    BUILDING  \
0       30075445                 MORRIS PARK BAKE SHOP     2  1007         
1       30112340                               WENDY'S     3         469   
2       30191841        DJ REYNOLDS PUB AND RESTAURANT     1         351   
3       40356483                    WILKEN'S FINE FOOD     3  7114         
4       30191841        DJ REYNOLDS PUB AND RESTAURANT     1         351   
5       30191841        DJ REYNOLDS PUB AND RESTAURANT     1         351   
6       40356018                       RIVIERA CATERER     3        2780   
7       40356068                    TOV KOSHER KITCHEN     4       97-22   
8       40356483                    WILKEN'S FINE FOOD     3  7114         
9       40356731           TASTE THE TROPICS ICE CREAM     3  1839         
10      40356483                    WILKEN'S FINE FOOD     3  7114         
11      40356649                       REGINA CATERERS     3        6409   
12      40356731           TASTE THE TROPICS ICE CREAM     3  1839         
13      40357217                             WILD ASIA     2        2300   
14      30112340                               WENDY'S     3         469   
15      40356442                         KOSHER ISLAND     5        2206   
16      40357217                             WILD ASIA     2        2300   
17      40361521                         GLORIOUS FOOD     1         522   
18      40356068                    TOV KOSHER KITCHEN     4       97-22   
19      40356151               BRUNOS ON THE BOULEVARD     4       88-25   
20      40357217                             WILD ASIA     2        2300   
21      40359480            1 EAST 66TH STREET KITCHEN     1  1            
22      40358429                       MAY MAY KITCHEN     3        1269   
23      40360045                           SEUDA FOODS     3  705          
24      30112340                               WENDY'S     3         469   
25      40356151               BRUNOS ON THE BOULEVARD     4       88-25   
26      40356442                         KOSHER ISLAND     5        2206   
27      40356442                         KOSHER ISLAND     5        2206   
28      40356649                       REGINA CATERERS     3        6409   
29      40357217                             WILD ASIA     2        2300   
...          ...                                   ...   ...         ...   
527619  50013562                            CHINA THAI     3         867   
527620  50013567                           ANDY'S CAFE     4       23-39   
527621  50013569                                   GFC     3         941   
527622  50013585                             BRUNSWICK     3         240   
527623  50013607  DELICIAS CALENAS BAKERY & RESTAURANT     4       95-41   
527624  50013609                      TASTE BUDDY CAFE     3        5810   
527625  50013616                            THAT STORE     4      135-28   
527626  50013755                    NINETY9 & UP DINER     4  187-44       
527627  50013777                               ALLADIN     3        2158   
527628  50013782                       NEW STAR BAKERY     4      41-21A   
527629  50013794                            FUEL FEVER     3        8615   
527630  50013827                      SNOWDAYS SHAVERY     1  241          
527631  50013871                    GREEN SUSHI BISTRO     3  104          
527632  50013900                              SUN HING     3        1167   
527633  50013937     CROWN FRIED CHICKEN & COFFEE SHOP     3        1604   
527634  50013941                        BAGEL ROAD INC     3        1424   
527635  50013976                           banana king     4       83-17   
527636  50013976                           banana king     4       83-17   
527637  50013993                 787 SOUTHERN BLVD LLC     2         787   
527638  50013998                 FAIR FOLKS AND A GOAT     1         330   
527639  50014098                         DUNKIN DONUTS     

In [402]:
v_and_c = []
for ic in pc_vl_new.index.values:
    vcode = pc_vl_new.ix[ic]['VIOLCODE']
    ccode = pc_vl_new.ix[ic]['CUISINECODE']
    v_and_c.append(sum(np.logical_and(tt1_ok['CUISINECODE'] == ccode, tt1_ok['VIOLCODE'] == vcode)))    
pc_vl_new['VOLI_CUIS_COUNT'] = v_and_c


In [421]:
pc_vl_new['VOLI_CUIS_PROB'] = pc_vl_new['VOLI_CUIS_COUNT'] / sum(pc_fl['count'])
pc_vl_new['FINAL_PROB'] = (pc_vl_new['VOLI_CUIS_PROB'] / pc_vl_new['CUISINE_PROB']) / pc_vl_new['VIOLATION_PROB']
pc_vl_new_filt = pc_vl_new[pc_vl_new['VOLI_CUIS_COUNT']>100]
pc_vl_new_filt = pc_vl_new_filt.sort('FINAL_PROB', ascending = False)
pc_vl_new_top20 = pc_vl_new_filt[:20]
print np.mean(pc_vl_new_top20['FINAL_PROB'])
pc_vl_new_top20

2.37274658413


count  VIOLATION_PROB VIOLCODE  CUISINECODE  CUISINE_PROB  \
413    4726        0.009094      04C           49      0.035286   
622    1945        0.003743      20D           14      0.028538   
2029   9264        0.017826      04A           51      0.005066   
1458   3536        0.006804      10E           29      0.012105   
1639   9264        0.017826      04A           43      0.007048   
1325   4364        0.008397      09B           82      0.013142   
1504  15954        0.030698      04H           47      0.008492   
356    3160        0.006080      02H           55      0.035373   
876    3160        0.006080      02H           44      0.015690   
90     4364        0.008397      09B           20      0.116916   
486    3160        0.006080      02H           17      0.030919   
938    3536        0.006804      10E           39      0.014716   
3577  45819        0.088164      02G           75      0.001106   
112     496        0.000954      04D           20      0.116916   
52      453        0.000872      15I            3      0.240925   
1454   4401        0.008468      22C           29      0.012105   
1769   9264        0.017826      04A           56      0.006362   
681    3160        0.006080      02H           77      0.025139   
918   21508        0.041385      04N           39      0.014716   
605    5860        0.011276      10A           14      0.028538   

      VOLI_CUIS_COUNT  VOLI_CUIS_PROB  FINAL_PROB  
413               541        0.001041    3.244136  
622               175        0.000337    3.152818  
2029              145        0.000279    3.089541  
1458              130        0.000250    3.037268  
1639              193        0.000371    2.955915  
1325              151        0.000291    2.632964  
1504              321        0.000618    2.369278  
356               260        0.000500    2.326052  
876               112        0.000216    2.258958  
90               1121        0.002157    2.197078  
486               206        0.000396    2.108402  
938               109        0.000210    2.094665  
3577              106        0.000204    2.091400  
112               121        0.000233    2.086548  
52                227        0.000437    2.079916  
1454              110        0.000212    2.064873  
1769              117        0.000225    1.985021  
681               151        0.000291    1.900857  
918               600        0.001155    1.895622  
605               315        0.000606    1.883620

In [415]:
pc_vl_new.loc[:20]

count  VIOLATION_PROB VIOLCODE  CUISINECODE  CUISINE_PROB  \
0   66420        0.127804      10F            3      0.240925   
1   49564        0.095370      08A            3      0.240925   
2   45819        0.088164      02G            3      0.240925   
3   38306        0.073708      04L            3      0.240925   
4   31331        0.060286      10B            3      0.240925   
5   30966        0.059584      06D            3      0.240925   
6   25370        0.048816      06C            3      0.240925   
7   24523        0.047187      02B            3      0.240925   
8   21508        0.041385      04N            3      0.240925   
9   15954        0.030698      04H            3      0.240925   
10  14568        0.028031      04M            3      0.240925   
11  13741        0.026440      06E            3      0.240925   
12  10016        0.019273      06F            3      0.240925   
13   9381        0.018051      06A            3      0.240925   
14   9264        0.017826      04A            3      0.240925   
15   8727        0.016792      10H            3      0.240925   
16   6806        0.013096      09C            3      0.240925   
17   6272        0.012068      04J            3      0.240925   
18   6240        0.012007      08C            3      0.240925   
19   5960        0.011468      05D            3      0.240925   
20   5860        0.011276      10A            3      0.240925   

    VOLI_CUIS_COUNT  VOLI_CUIS_PROB  FINAL_PROB  
0             16562        0.031868    1.034980  
1             11126        0.021408    0.931732  
2             10890        0.020954    0.986508  
3              8351        0.016069    0.904878  
4              8246        0.015867    1.092414  
5              9488        0.018257    1.271768  
6              5379        0.010350    0.880034  
7              3836        0.007381    0.649267  
8              5725        0.011016    1.104825  
9              5779        0.011120    1.503493  
10             2286        0.004399    0.651320  
11             3113        0.005990    0.940327  
12             2022        0.003891    0.837925  
13             1810        0.003483    0.800843  
14             2195        0.004224    0.983454  
15             2744        0.005280    1.305081  
16             1646        0.003167    1.003821  
17             1473        0.002834    0.974799  
18             1516        0.002917    1.008400  
19             1527        0.002938    1.063435  
20             1438        0.002767    1.018543

In [425]:
# ((cuisine, violation), ratio, count)
#[(("Café/Coffee/Tea", "Toilet facility not maintained and provided with toilet paper."), 1.87684775827172, 315)] * 20
total_count = 0
#zip_codes = []
#mean_scores = []
for fprob_idx in pc_vl_new_top20.index.values:
#    zip_codes.append(zipcode)
    count = pc_vl_new_top20.ix[fprob_idx]['VOLI_CUIS_COUNT']
    ratio = pc_vl_new_top20.ix[fprob_idx]['FINAL_PROB']
    cz_code = pc_vl_new_top20.ix[fprob_idx]['CUISINECODE']
    cz_name = czn.loc[czn['CUISINECODE']==cz_code,'CODEDESC'].values[0]
    violation_code = pc_vl_new_top20.ix[fprob_idx]['VIOLCODE']
    violation_name = vi.loc[vi['VIOLATIONCODE']==violation_code,'VIOLATIONDESC'].values[0]
    total_count += count
#    mean_scores.append(scores.mean())
    print("(('{cuis}', '{viol}'), {ratio}, {count}), ".format(cuis = cz_name, viol = violation_name,
                                                           ratio = ratio, count = count))
print total_count  

(('Japanese', 'Eggs found dirty cracked liquid frozen or powdered eggs not pasteurized.'), 3.24413628229, 541), 
(('Café/Coffee/Tea', '“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric) latex gloves sign not posted. Inspection report sign not posted.'), 3.15281790283, 175), 
(('Juice, Smoothies, Fruit Salads', 'Food from unapproved or unknown source spoiled adulterated or home canned.'), 3.08954068739, 145), 
(('Donuts', 'Accurate thermometer not provided in refrigerated or hot holding equipment.'), 3.037267501, 130), 
(('Ice Cream, Gelato, Yogurt, Ices', 'Food from unapproved or unknown source spoiled adulterated or home canned.'), 2.9559150772, 193), 
(('Thai', 'Thawing procedures improper.'), 2.63296399152, 151), 
(('Irish', 'Food in contact with utensil container or pipe that consist of toxic material.'), 2.36927766719, 321), 
(('Mexican', 'Food not cooled by an approve

In [397]:
cz_reps = []
cz_prob_reps = []
for cz in pc_cz_fl.index.values:
    cz_reps += [cz] * len(pc_fl)
    cz_prob_reps += [pc_cz_fl.ix[cz]['CUISINE_PROB']] * len(pc_fl)
cz_reps
cz_prob_reps

[0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.24092496072700176,
 0.2409249

In [428]:
ttt = [(('Japanese', 'Eggs found dirty cracked liquid frozen or powdered eggs not pasteurized.'), 3.24413628229, 541), (('Café/Coffee/Tea', '“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric) latex gloves sign not posted. Inspection report sign not posted.'), 3.15281790283, 175), (('Juice, Smoothies, Fruit Salads', 'Food from unapproved or unknown source spoiled adulterated or home canned.'), 3.08954068739, 145), (('Donuts', 'Accurate thermometer not provided in refrigerated or hot holding equipment.'), 3.037267501, 130), (('Ice Cream, Gelato, Yogurt, Ices', 'Food from unapproved or unknown source spoiled adulterated or home canned.'), 2.9559150772, 193), (('Thai', 'Thawing procedures improper.'), 2.63296399152, 151), (('Irish', 'Food in contact with utensil container or pipe that consist of toxic material.'), 2.36927766719, 321), (('Mexican', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140&#176;F to 70&#176;F or less within 2 hours and from 70&#176;F to 41&#176;F or less within 4 additional hours.'), 2.32605228091, 260), (('Indian', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140&#176;F to 70&#176;F or less within 2 hours and from 70&#176;F to 41&#176;F or less within 4 additional hours.'), 2.25895790403, 112), (('Chinese', 'Thawing procedures improper.'), 2.19707802972, 1121), (('Caribbean', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140&#176;F to 70&#176;F or less within 2 hours and from 70&#176;F to 41&#176;F or less within 4 additional hours.'), 2.10840246875, 206), (('Hamburgers', 'Accurate thermometer not provided in refrigerated or hot holding equipment.'), 2.09466545479, 109), (('Soups & Sandwiches', 'Cold food held above 41&#176;F (smoked fish above 38&#176;F) except during necessary preparation.'), 2.09139964377, 106), (('Chinese', 'Canned food product observed swollen leaking rusted severely dented.'), 2.08654763734, 121), (('American ', '&quot;Smoking Permitted&quot; and/or &quot;No Smoking&quot; sign not conspicuously posted. Health warning not present on &quot;Smoking Permitted&quot; sign.'), 2.07991629945, 227), (('Donuts', 'Bulb not shielded or shatterproof in areas where there is extreme heat temperature changes or where accidental contact may occur.'), 2.06487261145, 110), (('Middle Eastern', 'Food from unapproved or unknown source spoiled adulterated or home canned.'), 1.98502112889, 117), (('Spanish', 'Food not cooled by an approved method whereby the internal product temperature is reduced from 140&#176;F to 70&#176;F or less within 2 hours and from 70&#176;F to 41&#176;F or less within 4 additional hours.'), 1.90085711385, 151), (('Hamburgers', 'Evidence of mice or live mice present in facility.'), 1.89562161163, 600), (('Café/Coffee/Tea', 'Vermin or other live animal present in non-food area.'), 1.8836203887, 315)] 
ttt

[(('Japanese',
   'Eggs found dirty cracked liquid frozen or powdered eggs not pasteurized.'),
  3.24413628229,
  541),
 (('Caf\xc3\xa9/Coffee/Tea',
   '\xe2\x80\x9cChoking first aid\xe2\x80\x9d poster not posted. \xe2\x80\x9cAlcohol and pregnancy\xe2\x80\x9d warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric) latex gloves sign not posted. Inspection report sign not posted.'),
  3.15281790283,
  175),
 (('Juice, Smoothies, Fruit Salads',
   'Food from unapproved or unknown source spoiled adulterated or home canned.'),
  3.08954068739,
  145),
 (('Donuts',
   'Accurate thermometer not provided in refrigerated or hot holding equipment.'),
  3.037267501,
  130),
 (('Ice Cream, Gelato, Yogurt, Ices',
   'Food from unapproved or unknown source spoiled adulterated or home canned.'),
  2.9559150772,
  193),
 (('Thai', 'Thawing procedures improper.'), 2.63296399152, 151),
 (('Irish',
   'Food in contact with utensil container or pipe that consi

In [395]:
pp = pc_cz_fl.index.values[0]
pc_cz_fl.ix[pp]['CUISINE_PROB']

0.24092496072700176

In [381]:
pc_cz_fl.index.values

array([ 3, 20, 53, 62, 48, 55, 49, 17,  8, 14, 77, 63, 27, 44, 39,  5, 50,
       18, 35, 52, 82, 69, 29, 47, 54, 43,  7, 56, 72, 70, 81, 51, 38, 67,
       84, 61, 99,  2, 28, 78, 83, 30, 22, 73, 12, 23, 37, 59,  9, 10, 64,
       24, 13,  4, 80, 75, 34, 68, 21, 41, 60, 31, 32,  1, 33, 57, 66,  6,
       45, 76, 15, 71, 42,  0, 26, 46, 40, 58, 36, 25, 74, 11, 19, 16])

In [359]:
pc_cz = pd.DataFrame(tt1_ok.groupby('CUISINECODE').size())
pc_cz.columns=['count']
pc_cz_fl = pc_cz.sort('count', ascending = False)
pc_cz_fl['CUISINE_PROB'] = pc_cz_fl['count'] / sum(pc_cz_fl['count'])
pc_cz_fl

count  CUISINE_PROB
CUISINECODE                      
3            125453      0.240925
20            60880      0.116916
53            26063      0.050052
62            25686      0.049328
48            25637      0.049234
55            18419      0.035373
49            18374      0.035286
17            16100      0.030919
8             16100      0.030919
14            14860      0.028538
77            13090      0.025139
63            10645      0.020443
27             9590      0.018417
44             8170      0.015690
39             7663      0.014716
5              7654      0.014699
50             7532      0.014465
18             7442      0.014292
35             7409      0.014229
52             6957      0.013361
82             6843      0.013142
69             6510      0.012502
29             6303      0.012105
47             4422      0.008492
54             4169      0.008006
43             3670      0.007048
7              3657      0.007023
56             3313      0.006362
72             3245      0.006232
70             3057      0.005871
...             ...           ...
80              614      0.001179
75              576      0.001106
34              567      0.001089
68              552      0.001060
21              438      0.000841
41              433      0.000832
60              410      0.000787
31              348      0.000668
32              334      0.000641
1               300      0.000576
33              297      0.000570
57              284      0.000545
66              269      0.000517
6               248      0.000476
45              213      0.000409
76              203      0.000390
15              179      0.000344
71              149      0.000286
42              145      0.000278
0               131      0.000252
26              127      0.000244
46               79      0.000152
40               68      0.000131
58               59      0.000113
36               51      0.000098
25               49      0.000094
74               46      0.000088
11               37      0.000071
19                4      0.000008
16                2      0.000004

[84 rows x 2 columns]

In [393]:
pc_fl
    

count  VIOLATION_PROB
VIOLCODE                       
10F       66420        0.127804
08A       49564        0.095370
02G       45819        0.088164
04L       38306        0.073708
10B       31331        0.060286
06D       30966        0.059584
06C       25370        0.048816
02B       24523        0.047187
04N       21508        0.041385
04H       15954        0.030698
04M       14568        0.028031
06E       13741        0.026440
06F       10016        0.019273
06A        9381        0.018051
04A        9264        0.017826
10H        8727        0.016792
09C        6806        0.013096
04J        6272        0.012068
08C        6240        0.012007
05D        5960        0.011468
10A        5860        0.011276
16B        5852        0.011260
10D        4979        0.009580
04C        4726        0.009094
22C        4401        0.008468
09B        4364        0.008397
10I        3898        0.007500
20F        3775        0.007264
10E        3536        0.006804
04K        3306        0.006361
...         ...             ...
08B        2400        0.004618
99B        2257        0.004343
20D        1945        0.003743
22A        1746        0.003360
16A         916        0.001763
20A         884        0.001701
05H         844        0.001624
05F         827        0.001591
03A         593        0.001141
02C         550        0.001058
04O         541        0.001041
02A         498        0.000958
04D         496        0.000954
10C         479        0.000922
20E         476        0.000916
03C         474        0.000912
15J         464        0.000893
15I         453        0.000872
22B         396        0.000762
15K         391        0.000752
03B         331        0.000637
04E         324        0.000623
15S         298        0.000573
16C         188        0.000362
07A         155        0.000298
05E         151        0.000291
18D         137        0.000264
10G         115        0.000221
06G         114        0.000219
05A         111        0.000214

[65 rows x 2 columns]

In [ ]:
pc = pd.DataFrame(tt1.groupby('ZIPCODE').size())
pc.columns=['count']
pc_gt100 = pc[pc['count']>100].sort('count', ascending = False)
pc_gt100

In [12]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages')
from pysqlite2 import dbapi2 as sqlite3
con = sqlite3.connect('example.db')
c = con.cursor()
con.enable_load_extension(True)
con.load_extension("/home/vagrant/sql_setup/libsqlitefunctions.so")

In [11]:
import csv
c.execute("DROP TABLE IF EXISTS WEBEX;")
c.execute("CREATE TABLE WEBEX (CAMIS, DBA, BORO, BUILDING, STREET, ZIPCODE, PHONE, CUISINECODE, INSPDATE,\
                               ACTION, VIOLCODE, SCORE, CURRENTGRADE, GRADEDATE, RECORDDATE);")

#with open('WebExtract.txt','rb') as fin:
    # csv.DictReader uses first line in file for column headings by default
#    dr = csv.DictReader(fin) # comma is default delimiter
#    to_db = [(i['col1'], i['col2']) for i in dr]

#cur.executemany("INSERT INTO t (col1, col2) VALUES (?, ?);", to_db)
#con.commit()


with open('WebExtract.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    for row in reader:
        row_utf8 = [unicode(el, "utf8") for el in row]
        c.execute("INSERT INTO WEBEX (CAMIS, DBA, BORO, BUILDING, STREET, ZIPCODE, PHONE, CUISINECODE, \
                               INSPDATE, ACTION, VIOLCODE, SCORE, CURRENTGRADE, GRADEDATE, RECORDDATE) \
                   VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", row_utf8)
        con.commit()

KeyboardInterrupt: 

In [158]:
import csv
c.execute("DROP TABLE IF EXISTS WEBEX;")
c.execute("CREATE TABLE WEBEX (CAMIS, DBA, BORO, BUILDING, STREET, ZIPCODE, PHONE, CUISINECODE, INSPDATE,\
                               ACTION, VIOLCODE, SCORE, CURRENTGRADE, GRADEDATE, RECORDDATE);")

#with open('WebExtract.txt','rb') as fin:
    # csv.DictReader uses first line in file for column headings by default
#    dr = csv.DictReader(fin) # comma is default delimiter
#    to_db = [(i['col1'], i['col2']) for i in dr]

#cur.executemany("INSERT INTO t (col1, col2) VALUES (?, ?);", to_db)
#con.commit()


with open('WebExtract.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    to_db = []
    for row in reader:
        row_utf8 = tuple(unicode(el, "utf8") for el in row)
        to_db += row_utf8
    c.executemany("INSERT INTO WEBEX (CAMIS, DBA, BORO, BUILDING, STREET, ZIPCODE, PHONE, CUISINECODE, \
                                      INSPDATE, ACTION, VIOLCODE, SCORE, CURRENTGRADE, GRADEDATE, RECORDDATE) \
                   VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
    con.commit()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 15, and there are 5 supplied.

In [163]:
import pandas as pd
from pandas import datetime
import numpy as np
import re
tt = pd.read_csv('WebExtract.txt', names=['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 
                                          'PHONE', 'CUISINECODE', 'INSPDATE', 'ACTION', 'VIOLCODE',
                                          'SCORE', 'CURRENTGRADE', 'GRADEDATE', 'RECORDDATE'],
                dtype={'CAMIS': object, 'DBA': object, 'BORO': int, 'BUILDING': object, 
                       'STREET': object, 'ZIPCODE': int, 'PHONE': int, 'CUISINECODE': object,
                       'INSPDATE': datetime, 'ACTION': object, 'VIOLCODE': object, 'SCORE': int,
                       'CURRENTGRADE': object, 'GRADEDATE': datetime, 'RECORDDATE': datetime})

ValueError: cannot safely convert passed user dtype of <i8 for object dtyped data in column 2

In [180]:
tt1 = pd.read_csv('WebExtract.txt', names=['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 
                                          'PHONE', 'CUISINECODE', 'INSPDATE', 'ACTION', 'VIOLCODE',
                                          'SCORE', 'CURRENTGRADE', 'GRADEDATE', 'RECORDDATE'],
                 skiprows=1)

In [182]:
tt1.head()

CAMIS                             DBA  BORO    BUILDING  \
0  30075445           MORRIS PARK BAKE SHOP     2  1007         
1  30112340                         WENDY'S     3         469   
2  30191841  DJ REYNOLDS PUB AND RESTAURANT     1         351   
3  40356483              WILKEN'S FINE FOOD     3  7114         
4  30191841  DJ REYNOLDS PUB AND RESTAURANT     1         351   

                                              STREET  ZIPCODE       PHONE  \
0  MORRIS PARK AVE                               ...    10462  7188924968   
1                                    FLATBUSH AVENUE    11225  7182875005   
2                                     WEST 57 STREET    10019  2122452912   
3                   AVENUE U                            11234  7184443838   
4                                     WEST 57 STREET    10019  2122452912   

   CUISINECODE             INSPDATE ACTION VIOLCODE  SCORE CURRENTGRADE  \
0            8  2014-03-03 00:00:00      D      10F      2            A   
1           39  2014-07-01 00:00:00      F      06A     23            B   
2            3  2013-07-22 00:00:00      D      10B     11            A   
3           27  2014-05-29 00:00:00      D      08C     10            A   
4            3  2013-07-22 00:00:00      D      02G     11            A   

             GRADEDATE                     RECORDDATE  
0  2014-03-03 00:00:00  2014-09-04 06:01:28.403000000  
1  2014-07-01 00:00:00  2014-09-04 06:01:28.403000000  
2  2013-07-22 00:00:00  2014-09-04 06:01:28.403000000  
3  2014-05-29 00:00:00  2014-09-04 06:01:28.403000000  
4  2013-07-22 00:00:00  2014-09-04 06:01:28.403000000

In [198]:
tt1['ZIPCODE'] = tt1['ZIPCODE'].astype('int')

ValueError: Cannot convert NA to integer

# Zipcode

In [193]:
pc = pd.DataFrame(tt1.groupby('ZIPCODE').size())
pc.columns=['count']
pc_gt100 = pc[pc['count']>100].sort('count', ascending = False)
pc_gt100

count
ZIPCODE       
10003    14896
10019    13014
10036    12184
10013    12055
10002    10422
10022    10397
10011     9820
10016     9734
10014     9296
10001     9125
10012     8863
11354     8764
11372     8101
10017     8059
11220     8037
11215     7943
10018     7414
10009     7379
11211     7284
11201     7270
11368     6427
11373     5430
11101     5419
10025     5400
11385     5331
11209     5328
10010     5226
11217     4959
11355     4689
11377     4660
...        ...
10474      752
10020      733
10037      700
11413      693
11370      659
11004      654
10030      651
11426      638
11433      633
11436      621
11411      571
11360      557
11429      554
10039      549
11693      521
10307      440
10119      404
11427      388
11040      369
11363      362
11371      232
10280      222
10112      196
10282      189
11239      182
10281      159
11692      140
10044      136
10103      106
10121      103

[184 rows x 1 columns]

In [194]:
pc_gt100.index.values

array([ 10003.,  10019.,  10036.,  10013.,  10002.,  10022.,  10011.,
        10016.,  10014.,  10001.,  10012.,  11354.,  11372.,  10017.,
        11220.,  11215.,  10018.,  10009.,  11211.,  11201.,  11368.,
        11373.,  11101.,  10025.,  11385.,  11209.,  10010.,  11217.,
        11355.,  11377.,  11238.,  10028.,  10024.,  11226.,  11103.,
        11375.,  10023.,  10029.,  10458.,  11432.,  11222.,  11214.,
        10065.,  11223.,  10128.,  11235.,  11216.,  10021.,  11106.,
        11237.,  11218.,  10314.,  10467.,  11205.,  10038.,  10462.,
        10027.,  11249.,  11231.,  11229.,  10461.,  11358.,  11203.,
        10451.,  11208.,  10007.,  11207.,  11105.,  11204.,  11234.,
        10033.,  10463.,  11104.,  11435.,  11206.,  10004.,  11230.,
        10032.,  11219.,  10452.,  10468.,  11419.,  10034.,  11361.,
        11232.,  10472.,  10453.,  11225.,  10456.,  11236.,  10075.,
        10455.,  10031.,  11374.,  10466.,  11221.,  11213.,  10306.,
        10469.,  114

In [258]:
#(zipcode, mean score, standard error, number of violations)
#  [("11201", 20.3753782668501, 0.179295745965114, 7270)]
total_count = 0
zip_codes = []
mean_scores = []
for zipcode in pc_gt100.index.values:
    zip_codes.append(zipcode)
    count = pc_gt100.ix[zipcode]
    total_count += count
    scores_indices = tt1['ZIPCODE'] == zipcode
    scores = tt1.loc[scores_indices, 'SCORE']
    mean_scores.append(scores.mean())
    print("('{zipcode}', {score_m}, {score_se}, {count}), ".format(zipcode = int(zipcode), score_m = scores.mean(),
                                                               score_se = scores.std()/np.sqrt(sum(scores.notnull())),
                                                                   count = count[0]))
print total_count    
#    if i_deg < 100:
#        print("('{name}', {deg}), ".format(name = name, deg = deg))
#        i_deg += 1
#    else:
#        break

('10003', 22.4917491749, 0.127436382142, 14896), 
('10019', 21.4336348081, 0.129365691619, 13014), 
('10036', 22.4232513518, 0.144259363611, 12184), 
('10013', 23.8940681679, 0.15072846732, 12055), 
('10002', 24.2771071908, 0.172259295354, 10422), 
('10022', 21.9878905057, 0.155646122639, 10397), 
('10011', 21.3604105252, 0.142943528645, 9820), 
('10016', 22.781260167, 0.153239069092, 9734), 
('10014', 21.5263704559, 0.148723061824, 9296), 
('10001', 20.8615580507, 0.161723916171, 9125), 
('10012', 21.1203053065, 0.143473915026, 8863), 
('11354', 26.4620100986, 0.210474751761, 8764), 
('11372', 24.170565046, 0.195231029588, 8101), 
('10017', 22.4251012146, 0.169274457565, 8059), 
('11220', 25.8632254684, 0.198698321652, 8037), 
('11215', 21.4158576052, 0.16518420441, 7943), 
('10018', 22.9769000428, 0.18430456871, 7414), 
('10009', 21.9977889151, 0.178218529928, 7379), 
('11211', 21.0997010463, 0.172044944309, 7284), 
('11201', 21.9060928719, 0.179441607824, 7270), 
('11368', 23.540504

In [257]:
a=[]
a[0] = 5.5

IndexError: list assignment index out of range

# Borough

In [261]:
bo = pd.read_csv('Boroughs.txt')

In [262]:
bo[1]

BORO       BORONAME
0     1      MANHATTAN
1     2      THE BRONX
2     3       BROOKLYN
3     4         QUEENS
4     5  STATEN ISLAND

In [275]:
bo.loc[bo['BORO']==1,'BORONAME'][0]

'MANHATTAN'

In [265]:
pc = pd.DataFrame(tt1.groupby('BORO').size())
pc.columns=['count']
pc_order = pc.sort('count', ascending = False)
#pc_gt100

In [266]:
pc_order

count
BORO        
1     217231
3     125080
4     123494
2      48641
5      17386
0        103

In [276]:
pc_order.index.values

array([1, 3, 4, 2, 5, 0])

In [290]:
# (borough, mean score, stderr, number of violations)
#return [("MANHATTAN", 20.9020950048566, 0.0332666179191432, 217231)] * 5
total_count = 0
#zip_codes = []
#mean_scores = []
for boro in pc_order.index.values:
    if boro == 0:
        continue
#    zip_codes.append(zipcode)
    count = pc_order.ix[boro]
    boro_name = bo.loc[bo['BORO']==boro,'BORONAME'].values[0]
    total_count += count
    scores_indices = tt1['BORO'] == boro
    scores = tt1.loc[scores_indices, 'SCORE']
#    mean_scores.append(scores.mean())
    print("('{boroname}', {score_m}, {score_se}, {count}), ".format(boroname = boro_name, score_m = scores.mean(),
                                                               score_se = scores.std()/np.sqrt(sum(scores.notnull())),
                                                                   count = count[0]))
print total_count  

('MANHATTAN', 22.2355266058, 0.0332806903231, 217231), 
('BROOKLYN', 22.1690296494, 0.0443947480179, 125080), 
('QUEENS', 22.6854201601, 0.0456510461816, 123494), 
('THE BRONX', 21.5669020261, 0.0711722714718, 48641), 
('STATEN ISLAND', 20.9189491336, 0.107062958466, 17386), 
count    531832
Name: 1, dtype: int64


In [289]:
boro = pc_order.index.values[1]
bb = bo.loc[bo['BORO']==boro,'BORONAME'].values[0]

In [260]:
zipscore_dict = {'ZIPCODE' : zip_codes,
                 'mean_scores' : mean_scores}
zipscore_df = pd.DataFrame(zipscore_dict)
zipscore_df.to_csv('zipcode_scores_map.csv', index=False)

In [249]:
count[0]

8863

# Cuisine score

In [296]:
czn = pd.read_csv('Cuisine.txt')

In [297]:
czn

CUISINECODE                                           CODEDESC
0             2                                            African
1             3                                          American 
2             5                                              Asian
3            15                                              Cajun
4            17                                          Caribbean
5            20                                            Chinese
6            30                                   Eastern European
7            31                                           Egyptian
8            35                                             French
9            37                                             German
10           38                                              Greek
11           44                                             Indian
12           45                                         Indonesian
13           48                                            Italian
14           49                                           Japanese
15           50                                      Jewish/Kosher
16           52                                             Korean
17           53  Latin (Cuban, Dominican, Puerto Rican, South &...
18           54                                      Mediterranean
19           55                                            Mexican
20           56                                     Middle Eastern
21           67                                            Russian
22           76                                       Southwestern
23           82                                               Thai
24           99                                              Other
25            1                                             Afghan
26            4                                           Armenian
27            6                                         Australian
28            7                                    Bagels/Pretzels
29            8                                             Bakery
..          ...                                                ...
55           42                                   Hotdogs/Pretzels
56           43                    Ice Cream, Gelato, Yogurt, Ices
57           46                                            Iranian
58           47                                              Irish
59           51                     Juice, Smoothies, Fruit Salads
60           57                                           Moroccan
61           58                                 Nuts/Confectionary
62           59                                          Pakistani
63           60                                   Pancakes/Waffles
64           61                                           Peruvian
65           62                                              Pizza
66           63                                      Pizza/Italian
67           64                                             Polish
68           65                                         Polynesian
69           66                                         Portuguese
70           68                                             Salads
71           69                                         Sandwiches
72           70                     Sandwiches/Salads/Mixed Buffet
73           71                                       Scandinavian
74           72                                            Seafood
75           73                                          Soul Food
76           74                                              Soups
77           75                                 Soups & Sandwiches
78           77                                            Spanish
79           78                                              Steak
80           80                                              Tapas
81           81                                            Tex-Mex
82           83                                    

In [294]:
pc = pd.DataFrame(tt1.groupby('CUISINECODE').size())
pc.columns=['count']
pc_gt100 = pc[pc['count']>100].sort('count', ascending = False)[:75]
pc_gt100

count
CUISINECODE        
3            128372
20            61684
53            26489
62            26189
48            26152
55            18787
49            18647
17            16408
8             16395
14            15235
77            13349
63            10839
27             9731
44             8300
39             7868
5              7778
50             7624
18             7618
35             7576
52             7068
82             6947
69             6662
29             6449
47             4519
54             4249
43             3785
7              3726
56             3429
72             3308
70             3131
...             ...
23             1005
37              879
59              830
10              816
9               814
64              732
24              687
13              678
4               656
80              637
75              594
34              575
68              559
41              472
21              443
60              421
31              367
32              342
1               307
33              300
57              292
66              285
6               254
45              215
76              207
15              186
42              168
71              149
0               139
26              131

[75 rows x 1 columns]

In [295]:
print len(pc_gt100)

75


In [301]:
# (cuisine, mean score, stderr, number of violations)
#("French", 20.3550686378036, 0.17682605388627, 7576) *75
total_count = 0
#zip_codes = []
#mean_scores = []
for cz in pc_gt100.index.values:
#    zip_codes.append(zipcode)
    count = pc_gt100.ix[cz]
    cz_name = czn.loc[czn['CUISINECODE']==cz,'CODEDESC'].values[0]
    total_count += count
    scores_indices = tt1['CUISINECODE'] == cz
    scores = tt1.loc[scores_indices, 'SCORE']
#    mean_scores.append(scores.mean())
    print("('{czname}', {score_m}, {score_se}, {count}), ".format(czname = cz_name, score_m = scores.mean(),
                                                               score_se = scores.std()/np.sqrt(sum(scores.notnull())),
                                                                   count = count[0]))
print total_count  

('American ', 21.3544906965, 0.0417131594433, 128372), 
('Chinese', 24.9176143074, 0.0681657486193, 61684), 
('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 24.4995562727, 0.102777469959, 26489), 
('Pizza', 21.3703342279, 0.0917661808019, 26189), 
('Italian', 22.179524064, 0.0931603363183, 26152), 
('Mexican', 23.2664262821, 0.112626084034, 18787), 
('Japanese', 23.8085913181, 0.114477234114, 18647), 
('Caribbean', 23.0925961849, 0.126494730141, 16408), 
('Bakery', 23.065428995, 0.129825669748, 16395), 
('Café/Coffee/Tea', 17.1255183077, 0.103456335774, 15235), 
('Spanish', 23.4485958877, 0.13941442217, 13349), 
('Pizza/Italian', 22.1929531848, 0.139750900478, 10839), 
('Delicatessen', 24.5195100802, 0.162354928494, 9731), 
('Indian', 25.0934887256, 0.202403464671, 8300), 
('Hamburgers', 17.5123523093, 0.132404509807, 7868), 
('Asian', 26.0572285052, 0.21557599197, 7778), 
('Jewish/Kosher', 22.6756348662, 0.173425392707, 7624), 
('Chicken', 19.3176002275, 0.1534112

In [ ]:
len()

In [13]:
c.execute("DROP TABLE IF EXISTS ACTION;")
c.execute("CREATE TABLE ACTION (STARTDATE, ENDDATE, ACTIONCODE, ACTIONDESC);")

with open('Action.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    for row in reader:
        row_utf8 = [unicode(el, "utf8") for el in row]
        c.execute("INSERT INTO ACTION (STARTDATE, ENDDATE, ACTIONCODE, ACTIONDESC) \
                   VALUES(?, ?, ?, ?);", row_utf8)
        con.commit()

In [14]:
c.execute("DROP TABLE IF EXISTS CUISINE;")
c.execute("CREATE TABLE CUISINE (CUISINECODE, CODEDESC);")

with open('Cuisine.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    for row in reader:
        row_utf8 = [unicode(el, "utf8") for el in row]
        c.execute("INSERT INTO CUISINE (CUISINECODE, CODEDESC) \
                   VALUES(?, ?);", row_utf8)
        con.commit()

In [155]:
import pandas as pd
import numpy as np
import re
tt = pd.read_csv('Violation.txt', names=['STARTDATE', 'ENDDATE', 'CRITICALFLAG', 'VIOLATIONCODE', 'VIOLATIONDESC',
                                         'e1', 'e2', 'e3', 'e4'])
tt_new = tt.copy()
tt_new = tt_new.drop(['e1', 'e2', 'e3', 'e4'], axis=1)
min_error_col_ix = 5
max_error_col_ix = 8
for i in tt.index:
    if any(tt.loc[i,'e1':].notnull()):
        error_col_ix = min_error_col_ix
        pb_line = tt.ix[i, error_col_ix-1] + ',' + tt.ix[i, error_col_ix]
        error_col_ix += 1
        while error_col_ix <= max_error_col_ix and type(tt.ix[i, error_col_ix]) is str:
            pb_line += ',' + tt.ix[i, error_col_ix]
            error_col_ix += 1
        pb_line_noquotes = pb_line.replace('"','')
        clean_line = re.sub('<.*?>', '', pb_line_noquotes).replace(",",'').replace(";", ' ')
        clean_line_goodsymbols = re.sub('&#176;?', ' degrees ', clean_line)
        clean_line_bettersymbols = re.sub('&', ' and ', clean_line_goodsymbols)
        cleaner_line = re.sub('\s+', ' ', clean_line_goodsymbols).strip()
        tt_new.ix[i, min_error_col_ix-1] = cleaner_line
for i in tt_new.index:
    pot_comma_descr = tt_new.ix[i, min_error_col_ix-1]
    no_comma_descr = pot_comma_descr.replace(',',' ')
    no_comma = re.sub('\s+', ' ', no_comma_descr).strip()
    tt_new.ix[i, min_error_col_ix-1] = no_comma
tt_new.to_csv('Violation_corrected.txt', index=False)

In [153]:
tt_new

STARTDATE              ENDDATE  CRITICALFLAG  VIOLATIONCODE  \
0              STARTDATE              ENDDATE  CRITICALFLAG  VIOLATIONCODE   
1    1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01A   
2    2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01A   
3    2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01A   
4    2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01A   
5    2008-07-01 00:00:00  2009-08-01 00:00:00             Y            01A   
6    1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01B   
7    2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01B   
8    2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01B   
9    2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01B   
10   2008-07-01 00:00:00  2009-08-01 00:00:00             Y            01B   
11   1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01C   
12   2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01C   
13   2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01C   
14   2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01C   
15   2008-07-01 00:00:00  2009-08-01 00:00:00             Y            01C   
16   1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01D   
17   2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01D   
18   2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01D   
19   2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01D   
20   2008-07-01 00:00:00  2009-08-01 00:00:00             Y            01D   
21   1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01E   
22   2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01E   
23   2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01E   
24   2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01E   
25   2008-07-01 00:00:00  2009-08-01 00:00:00             Y            01E   
26   1901-01-01 00:00:00  2003-03-23 00:00:00             Y            01F   
27   2003-03-24 00:00:00  2005-02-17 00:00:00             Y            01F   
28   2005-02-18 00:00:00  2007-06-30 00:00:00             Y            01F   
29   2007-07-01 00:00:00  2008-06-30 00:00:00             Y            01F   
..                   ...                  ...           ...            ...   
690  2010-07-26 00:00:00  2099-12-31 00:00:00             N            16F   
691  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18A   
692  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18A   
693  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18B   
694  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18B   
695  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18C   
696  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18C   
697  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18D   
698  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18D   
699  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18E   
700  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18E   
701  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18F   
702  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18F   
703  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18G   
704  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18G   
705  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18H   
706  2010-07-26 00:00:00  2099-12-31 00:00:00             N            18H   
707  2009-08-02 00:00:00  2010-07-25 00:00:00             N            18I   
708  2010-07-26 00:00:00  2099-12-31 00:00:00             N            20A   
709  2010-07-26 00:00:00  2099-12-31 

In [157]:
c.execute("DROP TABLE IF EXISTS VIOLATION;")
c.execute("CREATE TABLE VIOLATION (STARTDATE, ENDDATE, CRITICALFLAG, VIOLATIONCODE, VIOLATIONDESC);")

with open('Violation_corrected.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    for row in reader:
        row_utf8 = [unicode(el, "utf8") for el in row]
        c.execute("INSERT INTO VIOLATION (STARTDATE, ENDDATE, CRITICALFLAG, VIOLATIONCODE, VIOLATIONDESC) \
                   VALUES(?, ?, ?, ?, ?);", row_utf8)
        con.commit()

In [ ]:
c.execute("DROP TABLE IF EXISTS BOROUGHS;")
c.execute("CREATE TABLE BOROUGHS (BORO, BORONAME);")

with open('Boroughs.txt','rb') as f: # CSV file input
    reader = csv.reader(f, delimiter=',', quotechar='"') # no header information with delimiter
    for row in reader:
        row_utf8 = [unicode(el, "utf8") for el in row]
        c.execute("INSERT INTO BOROUGHS (BORO, ENDDATE, BORONAME) \
                   VALUES(?, ?, ?, ?, ?);", row_utf8)
        con.commit()